In [69]:
import os, math, numpy as np, pandas as pd, torch
import torch.nn.functional as F
from dataclasses import dataclass
from typing import Dict, Tuple
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from torch_geometric.nn.models import DeepGraphInfomax

In [70]:
SEED    = 7
N       = 80   # số user
K       = 6    # số AP Wi-Fi
Fs       = 10   # số băng con
AREA    = 500.0  # m, vùng vuông [0,L]x[0,L]
NOISE_W = 1e-13  # W
PMIN, PMAX = 50e-3, 200e-3  # W
SL, SW = 1.2, 1.0           # hệ số hài lòng (chỉ dùng trong gen)
PRICE_C = 1.0               # giá cước LTE (chỉ dùng trong gen)

In [71]:
rng = np.random.default_rng(SEED)
torch.manual_seed(SEED)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Data

In [73]:
class StandardScalerNP:
    def __init__(self): self.mean_, self.std_ = None, None
    def fit(self, X): 
        self.mean_ = X.mean(0); self.std_ = X.std(0); self.std_[self.std_==0] = 1.0
    def transform(self, X): return (X - self.mean_) / self.std_
    def fit_transform(self, X): self.fit(X); return self.transform(X)


In [74]:
def pathloss_gain(d_m: np.ndarray, pl_ref_gain_db=-30.0, n=3.2, d0=1.0):
    """Return |h|^2 large-scale: PL(dB)=PL(d0)+10n log10(d/d0); gain_lin=10^(-PL/10)"""
    d = np.maximum(d_m, 1.0)
    PL_dB = pl_ref_gain_db + 10.0*n*np.log10(d/d0)
    return 10**(-PL_dB/10.0)

In [75]:
def generate_dataset(N=80, K=6, F=10, L=500.0, noise=1e-13,
                     pmin=50e-3, pmax=200e-3, SL=1.2, SW=1.0, c=1.0, seed=7):
    rng = np.random.default_rng(seed)
    # vị trí
    users = rng.uniform(0, L, (N,2))
    aps   = rng.uniform(0, L, (K,2))
    bs    = np.array([L/2, L/2])

    # kênh lớn + Rayleigh
    d_iL  = np.linalg.norm(users - bs, axis=1)
    g_iL  = pathloss_gain(d_iL) * rng.exponential(1.0, N)
    d_ik  = np.linalg.norm(users[:,None,:] - aps[None,:,:], axis=2)
    g_ik  = pathloss_gain(d_ik) * rng.exponential(1.0, (N,K))

    # tham số user
    Ci   = rng.uniform(0.0, 8.0, N)
    Lthr = rng.uniform(0.3, 0.8, N)
    Pmax = rng.uniform(pmin, pmax, N)
    P    = rng.uniform(0.5, 1.0, N) * Pmax

    # gán LTE/Wi-Fi (heuristic affordability + utility)
    snr_L = (0.5*(pmin+pmax))*g_iL/noise
    snr_W_best = (0.5*(pmin+pmax))*np.max(g_ik, axis=1)/noise
    uL = SL*np.log2(1+snr_L)
    uW = SW*np.log2(1+snr_W_best)
    x_LTE = (uL>=uW) & (c*np.log2(1+snr_L) <= Ci)
    ap_idx = np.argmax(g_ik, axis=1)
    ap_idx[x_LTE] = -1

    # subband greedy theo giảm nhiễu
    Psi = np.zeros((N,F), dtype=int)
    # hbar_ij: kênh từ j -> máy thu của i
    hbar = np.zeros((N,N))
    for i in range(N):
        if x_LTE[i]: hbar[i,:] = g_iL[:]          # j -> BS
        else:        hbar[i,:] = g_ik[:, ap_idx[i]]
    np.fill_diagonal(hbar, 0.0)

    users_on_f = [[] for _ in range(F)]
    order = rng.permutation(N)
    for i in order:
        best_f, best_den = None, None
        for f in range(F):
            js = users_on_f[f]
            den = (P[js]*hbar[i,js]).sum() + noise
            if best_den is None or den < best_den:
                best_den, best_f = den, f
        Psi[i,best_f] = 1
        users_on_f[best_f].append(i)

    # SINR & rate (để kiểm soát chất lượng dữ liệu; không dùng làm label)
    h_eff = np.where(x_LTE, g_iL, g_ik[np.arange(N), np.maximum(ap_idx,0)])
    SINR = np.zeros(N)
    for f in range(F):
        ids = np.where(Psi[:,f]==1)[0]
        if ids.size==0: continue
        den = hbar[np.ix_(ids,ids)].dot(P[ids]) + noise
        num = P[ids]*h_eff[ids]
        SINR[ids] += num/den
    rate = np.log2(1+SINR)

    # DataFrame users
    df = pd.DataFrame({
        "user_id": np.arange(N),
        "x_LTE": x_LTE.astype(int),
        "ap_idx": ap_idx.astype(int),
        "subband": Psi.argmax(1).astype(int),
        "P_max_W": Pmax, "P_W": P, "C_i": Ci, "L_thr": Lthr,
        "g_iL": g_iL, "SINR": SINR, "rate": rate
    })
    for k in range(K): df[f"g_i{k}"] = g_ik[:,k]

    return {
        "users": df,
        "aps_xy": aps,
        "bs_xy": bs,
        "noise": noise
    }


# Train Test Loader

In [76]:
data_gen = generate_dataset(N=N, K=K, F=Fs, L=AREA, noise=NOISE_W,
                            pmin=PMIN, pmax=PMAX, SL=SL, SW=SW, c=PRICE_C, seed=SEED)
df = data_gen["users"]
print(df.head())
print("N users:", len(df), "| K APs:", K, "| F subbands:", F)

   user_id  x_LTE  ap_idx  subband   P_max_W       P_W       C_i     L_thr  \
0        0      0       4        9  0.069362  0.043642  1.298483  0.451962   
1        1      0       1        5  0.153961  0.124950  3.646005  0.680345   
2        2      0       4        6  0.171822  0.094295  5.294045  0.487091   
3        3      0       0        8  0.167983  0.140216  4.062957  0.461594   
4        4      0       3        4  0.117861  0.115636  1.323182  0.618151   

           g_iL       SINR      rate          g_i0      g_i1      g_i2  \
0  2.401206e-05   1.562836  1.357741  1.777937e-05  0.000003  0.000004   
1  1.294232e-05  24.728199  4.685279  1.184890e-07  0.002254  0.000023   
2  4.236372e-07  77.753584  6.299274  1.920289e-03  0.000002  0.000006   
3  4.277738e-06   1.749671  1.459259  1.858029e-04  0.000002  0.000003   
4  1.927266e-04   6.078042  2.823350  2.709808e-05  0.000054  0.000299   

       g_i3          g_i4          g_i5  
0  0.000204  2.411748e-04  4.591673e-05  
1 

In [77]:
N = len(df)
g_L = df["g_iL"].to_numpy(np.float32)
P   = df["P_W"].to_numpy(np.float32)
ap  = df["ap_idx"].to_numpy(np.int64)
xL  = df["x_LTE"].to_numpy(np.int64)
sub = df["subband"].to_numpy(np.int64)

g_cols = [c for c in df.columns if c.startswith("g_i") and c!="g_iL"]
g_cols_sorted = sorted(g_cols, key=lambda c: int(c.replace("g_i","")))
g_ik = df[g_cols_sorted].to_numpy(np.float32) if g_cols_sorted else np.zeros((N,0), np.float32)
K = g_ik.shape[1]
noise = data_gen["noise"]

# hbar_ij: j -> máy thu của i
hbar = np.zeros((N,N), dtype=np.float32)
for i in range(N):
    if xL[i]==1: hbar[i,:] = g_L[:]                             # j -> BS
    else:
        kstar = ap[i]
        hbar[i,:] = g_ik[:, kstar] if 0 <= kstar < K else g_L[:]
np.fill_diagonal(hbar, 0.0)

# Cạnh có hướng giữa các user cùng băng con; trọng số ~ P_j * hbar_{ij}
edge_src, edge_dst, edge_w = [], [], []
scale = (P[:,None]*hbar).mean() + 1e-13

for f in np.unique(sub):
    ids = np.where(sub==f)[0]
    if ids.size <= 1: 
        continue
    for a in ids:
        for b in ids:
            if a == b: 
                continue
            w = (P[b] * hbar[a,b]) / scale
            # thưởng nhẹ nếu cùng điểm thu (cùng LTE hoặc cùng AP)
            same_recv = (xL[a]==1 and xL[b]==1) or (ap[a]>=0 and ap[a]==ap[b]>=0)
            if same_recv: 
                w += 0.1
            edge_src.append(a); edge_dst.append(b); edge_w.append(float(w))

In [78]:
edge_index = torch.tensor([edge_src, edge_dst], dtype=torch.long)
edge_weight = torch.tensor(edge_w, dtype=torch.float32)

# Features cho unsupervised: lấy mọi cột số an toàn (trừ id/meta)
drop_cols = {"user_id"}
feat_cols = [c for c in df.columns if c not in drop_cols]
X_np = df[feat_cols].to_numpy(np.float32)

# Chuẩn hoá feature (không có label)
scaler = StandardScalerNP()
X_np = scaler.fit_transform(X_np)

x = torch.tensor(X_np, dtype=torch.float32)

# Chia mask train/val/test (tùy hứng để quan sát loss/đánh giá phụ; unsupervised thực tế không bắt buộc)
perm = rng.permutation(N)
n_train = int(0.8 * N)
train_mask = torch.zeros(N, dtype=torch.bool); train_mask[perm[:n_train]] = True
val_mask   = torch.zeros(N, dtype=torch.bool); val_mask[perm[n_train:]] = True
test_mask  = torch.zeros(N, dtype=torch.bool); test_mask[:] = False

In [79]:
data = Data(
    x=x.to(device),
    edge_index=edge_index.to(device),
    edge_weight=edge_weight.to(device),
    train_mask=train_mask.to(device),
    val_mask=val_mask.to(device),
    test_mask=test_mask.to(device)
)
print(data)
print(f"Edges: {edge_index.size(1)} | Feature dim: {data.x.size(-1)}")

Data(x=[80, 16], edge_index=[2, 640], edge_weight=[640], train_mask=[80], val_mask=[80], test_mask=[80])
Edges: 640 | Feature dim: 16


# Model

In [80]:
class GCNEncoder(torch.nn.Module):
    def __init__(self, in_dim, hidden=128):
        super().__init__()
        self.conv1 = GCNConv(in_dim, hidden)
        self.conv2 = GCNConv(hidden, hidden)

    def forward(self, x, edge_index, edge_weight=None):
        x = self.conv1(x, edge_index, edge_weight=edge_weight)
        x = F.relu(x)
        x = self.conv2(x, edge_index, edge_weight=edge_weight)
        return x

In [81]:
def corruption(x, edge_index, edge_weight=None):
    perm = torch.randperm(x.size(0), device=x.device)
    x_cor = x[perm]
    return x_cor, edge_index, edge_weight

In [82]:
in_dim = data.x.size(-1)
HIDDEN = 128
encoder = GCNEncoder(in_dim, hidden=HIDDEN).to(device)

In [83]:
model = DeepGraphInfomax(
    hidden_channels=HIDDEN,
    encoder=encoder,
    summary=lambda z, *args, **kwargs: torch.sigmoid(z.mean(dim=0)),
    corruption=corruption
).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-3, weight_decay=1e-3)

In [86]:
EPOCHS = 500
logs = []
for epoch in range(1, EPOCHS+1):
    model.train()
    optimizer.zero_grad()
    pos_z, neg_z, summary = model(data.x, data.edge_index, edge_weight=data.edge_weight)
    loss = model.loss(pos_z, neg_z, summary)
    loss.backward()
    optimizer.step()

    # log mỗi epoch
    print(f"Epoch {epoch:03d} | DGI Loss {loss.item():.6f}")
    logs.append({"epoch": epoch, "dgi_loss": float(loss.item())})

Epoch 001 | DGI Loss 0.694165
Epoch 002 | DGI Loss 0.678621
Epoch 003 | DGI Loss 0.839534
Epoch 004 | DGI Loss 0.745292
Epoch 005 | DGI Loss 0.731227
Epoch 006 | DGI Loss 0.768462
Epoch 007 | DGI Loss 0.652986
Epoch 008 | DGI Loss 0.663310
Epoch 009 | DGI Loss 0.619599
Epoch 010 | DGI Loss 0.693337
Epoch 011 | DGI Loss 0.815674
Epoch 012 | DGI Loss 0.757255
Epoch 013 | DGI Loss 0.692985
Epoch 014 | DGI Loss 0.718283
Epoch 015 | DGI Loss 0.767184
Epoch 016 | DGI Loss 0.778644
Epoch 017 | DGI Loss 0.779975
Epoch 018 | DGI Loss 0.838607
Epoch 019 | DGI Loss 0.813146
Epoch 020 | DGI Loss 1.050694
Epoch 021 | DGI Loss 0.753130
Epoch 022 | DGI Loss 0.897526
Epoch 023 | DGI Loss 0.627854
Epoch 024 | DGI Loss 0.809447
Epoch 025 | DGI Loss 1.428406
Epoch 026 | DGI Loss 0.927996
Epoch 027 | DGI Loss 0.848322
Epoch 028 | DGI Loss 0.888756
Epoch 029 | DGI Loss 0.646029
Epoch 030 | DGI Loss 0.820853
Epoch 031 | DGI Loss 0.762035
Epoch 032 | DGI Loss 0.880531
Epoch 033 | DGI Loss 0.682202
Epoch 034 

In [85]:
model.eval()
with torch.no_grad():
    Z = model.encoder(data.x, data.edge_index, edge_weight=data.edge_weight)  # (N, HIDDEN)
print("Embeddings shape:", tuple(Z.shape))

Embeddings shape: (80, 128)
